In [1]:
"""
This tutorial shows how to generate adversarial examples
using JSMA in white-box setting.
The original paper can be found at:
https://arxiv.org/abs/1511.07528
"""
# pylint: disable=missing-docstring
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import numpy as np
from six.moves import xrange
import tensorflow as tf

from cleverhans.attacks import FastGradientMethod
from cleverhans.loss import CrossEntropy
from cleverhans.attacks import CarliniWagnerL2
from cleverhans.attacks import SaliencyMapMethod
from cleverhans.attacks import DeepFool
from cleverhans.utils import other_classes, set_log_level
from cleverhans.utils import pair_visual, grid_visual, AccuracyReport
from cleverhans.utils_tf import model_eval, model_argmax

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from cleverhans_model import CleverhansModel
from model import Model as My_model

from tensorflow.python import pywrap_tensorflow
import math
from random import choice

import matplotlib.pyplot as plt


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
model_dir = "./models/nat"

num_examples = 10
batch_size = 2
num_batches = int(math.ceil(num_examples / batch_size))

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
x_test = mnist.test.images
y_test = mnist.test.labels

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def classify(sess, img):
    x = tf.placeholder(tf.float32, shape = [None, 784])
    c_model.fprop(x)
    return sess.run(c_model.get_pred(), feed_dict = {x: img})
def plot(img):
    plt.imshow(np.resize(img,[28,28]), cmap='Greys_r')

In [4]:
def init_graph(model_dir):
    tf.reset_default_graph()
    # print(scope_vars)
    # x = tf.placeholder(tf.float32, shape = [None, 784])

    model = My_model()
    c_model = CleverhansModel('CNN', 10, model)
    checkpoint = tf.train.latest_checkpoint(model_dir)
    reader=pywrap_tensorflow.NewCheckpointReader(checkpoint)
    saver = tf.train.Saver()
    config = tf.ConfigProto()
    config.log_device_placement=False
    config.allow_soft_placement=True
    config.gpu_options.allow_growth=True
    session = tf.Session(config=config)
    session.run(tf.global_variables_initializer())
    saver.restore(session, checkpoint)
    return session, c_model

def batch_attack(data, attack, **params):
    x_adv = []
    for ibatch in range(num_batches):
        bstart = ibatch * batch_size
        bend = min(bstart + batch_size, num_examples)
        print('batch size: {}'.format(bend - bstart))

        x_batch = data[bstart:bend, :]

        x_batch_adv = attack.generate_np(x_batch, **params)
        
        x_adv = x_adv + x_batch_adv.tolist()
        
    return x_adv

def save_npy(algm, x_adv, filename = '/adv.npy'):
    path = './adv_test/' + algm + filename
    x_adv = np.concatenate(x_adv, axis=0)
    np.save(path, x_adv)
    

In [5]:
session, c_model = init_graph(model_dir)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./models/nat/checkpoint-7800


In [6]:
###########################################################################
# Craft adversarial examples using the Jacobian-based saliency map approach
###########################################################################

# y = tf.placeholder(tf.int64, shape = [None])

print('Crafting ' + str(num_examples) +
    ' adversarial examples')

jsma = SaliencyMapMethod(c_model, sess=session)
jsma_params = {'theta': 1., 'gamma': 0.1,
             'clip_min': 0., 'clip_max': 1.,
             'y_target': None}

x_adv = [] # adv accumulator

for sample_ind in xrange(0, num_examples):
    print('--------------------------------------')
    print('Attacking input %i/%i' % (sample_ind + 1, num_examples))
    sample = x_test[sample_ind:(sample_ind + 1)]
    adv = jsma.generate_np(sample, **jsma_params)
    x_adv.append(adv)
    
save_npy('jsma', x_adv)
session.close()

[INFO 2019-05-06 09:54:49,525 cleverhans] Constructing new graph for attack SaliencyMapMethod


Crafting 10 adversarial examples
--------------------------------------
Attacking input 1/10
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
--------------------------------------
Attacking input 2/10
--------------------------------------
Attacking input 3/10
--------------------------------------
Attacking input 4/10
--------------------------------------
Attacking input 5/10
--------------------------------------
Attacking input 6/10
--------------------------------------
Attacking input 7/10
--------------------------------------
Attacking input 8/10
-----

In [7]:
session, c_model = init_graph(model_dir)

INFO:tensorflow:Restoring parameters from ./models/nat/checkpoint-7800


In [8]:
LEARNING_RATE = .001
CW_LEARNING_RATE = .2
ATTACK_ITERATIONS = 100

print('Iterating over {} batches'.format(num_batches))

# x = tf.placeholder(tf.float32, shape = [None, 784])
# y = tf.placeholder(tf.int64, shape = [None])

###########################################################################
# Craft adversarial examples using Carlini and Wagner's approach
###########################################################################

print('Crafting ' + str(num_examples) + ' adversarial examples')
print("This could take some time ...")

# Instantiate a CW attack object
cw = CarliniWagnerL2(c_model, sess=session)

cw_params = {'binary_search_steps': 1,
               "y_target": None,
               'max_iterations': ATTACK_ITERATIONS,
               'learning_rate': CW_LEARNING_RATE,
               'batch_size': batch_size,
               'initial_const': 10}

x_adv = batch_attack(x_test, cw, **cw_params)
save_npy('cw', x_adv)
session.close()

[INFO 2019-05-06 09:54:59,564 cleverhans] Constructing new graph for attack CarliniWagnerL2


Iterating over 5 batches
Crafting 10 adversarial examples
This could take some time ...
batch size: 2
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
batch size: 2
batch size: 2
batch size: 2
batch size: 2


In [11]:
session, c_model = init_graph(model_dir)

INFO:tensorflow:Restoring parameters from ./models/nat/checkpoint-7800


In [12]:
epsilon = [0.1,0.2,0.3]
for i in range(3):
    fgsm_params = {
      'eps': epsilon[i],
      'clip_min': 0.,
      'clip_max': 1.
    }

    # Set TF random seed to improve reproducibility
    tf.set_random_seed(2333)

    # x = tf.placeholder(tf.float32, shape = [None, 784])
    # y = tf.placeholder(tf.int64, shape = [None])

    ###########################################################################
    # Craft adversarial examples using FGSM
    ###########################################################################

    print('Crafting ' + str(num_examples) + ' adversarial examples')

    fgsm = FastGradientMethod(c_model, sess=session)
    x_adv = batch_attack(x_test, fgsm, **fgsm_params)
    save_npy('fgsm/0' + str(i+1) + '/', x_adv)
session.close()

[INFO 2019-05-06 09:59:12,515 cleverhans] Constructing new graph for attack FastGradientMethod


Crafting 10 adversarial examples
batch size: 2


[INFO 2019-05-06 09:59:12,958 cleverhans] Constructing new graph for attack FastGradientMethod


batch size: 2
batch size: 2
batch size: 2
batch size: 2
Crafting 10 adversarial examples
batch size: 2


[INFO 2019-05-06 09:59:13,407 cleverhans] Constructing new graph for attack FastGradientMethod


batch size: 2
batch size: 2
batch size: 2
batch size: 2
Crafting 10 adversarial examples
batch size: 2
batch size: 2
batch size: 2
batch size: 2
batch size: 2


In [13]:
session, c_model = init_graph(model_dir)

INFO:tensorflow:Restoring parameters from ./models/nat/checkpoint-7800


In [14]:

###########################################################################
# Craft adversarial examples using FGSM
###########################################################################

print('Crafting ' + str(num_examples) + ' adversarial examples')

deepfool = DeepFool(c_model, sess=session)
x_adv = batch_attack(x_test, deepfool)
save_npy('deepfool', x_adv)

session.close()

[INFO 2019-05-06 09:59:16,079 cleverhans] Constructing new graph for attack DeepFool


Crafting 10 adversarial examples
batch size: 2


[INFO 2019-05-06 09:59:17,820 cleverhans] Attack result at iteration 5 is [7 2]
[INFO 2019-05-06 09:59:17,877 cleverhans] Attack result at iteration 10 is [7 2]
[INFO 2019-05-06 09:59:17,914 cleverhans] Attack result at iteration 13 is [3 1]
[INFO 2019-05-06 09:59:17,915 cleverhans] 2 out of 2 become adversarial examples at iteration 13
[INFO 2019-05-06 09:59:17,971 cleverhans] Attack result at iteration 5 is [1 0]
[INFO 2019-05-06 09:59:18,025 cleverhans] Attack result at iteration 10 is [1 0]
[INFO 2019-05-06 09:59:18,076 cleverhans] Attack result at iteration 15 is [8 0]
[INFO 2019-05-06 09:59:18,108 cleverhans] Attack result at iteration 18 is [8 2]
[INFO 2019-05-06 09:59:18,109 cleverhans] 2 out of 2 become adversarial examples at iteration 18


batch size: 2
batch size: 2


[INFO 2019-05-06 09:59:18,166 cleverhans] Attack result at iteration 5 is [4 1]
[INFO 2019-05-06 09:59:18,221 cleverhans] Attack result at iteration 10 is [9 1]
[INFO 2019-05-06 09:59:18,242 cleverhans] Attack result at iteration 12 is [9 7]
[INFO 2019-05-06 09:59:18,243 cleverhans] 2 out of 2 become adversarial examples at iteration 12
[INFO 2019-05-06 09:59:18,299 cleverhans] Attack result at iteration 5 is [4 9]
[INFO 2019-05-06 09:59:18,353 cleverhans] Attack result at iteration 10 is [8 9]
[INFO 2019-05-06 09:59:18,374 cleverhans] Attack result at iteration 12 is [8 4]
[INFO 2019-05-06 09:59:18,375 cleverhans] 2 out of 2 become adversarial examples at iteration 12
[INFO 2019-05-06 09:59:18,434 cleverhans] Attack result at iteration 5 is [5 9]


batch size: 2
batch size: 2


[INFO 2019-05-06 09:59:18,487 cleverhans] Attack result at iteration 10 is [5 7]
[INFO 2019-05-06 09:59:18,520 cleverhans] Attack result at iteration 13 is [6 7]
[INFO 2019-05-06 09:59:18,521 cleverhans] 2 out of 2 become adversarial examples at iteration 13
